In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import cast

from huggingface_hub import list_models
from rich import print as rprint
from transformers import AutoTokenizer, TextStreamer, pipeline
from transformers.pipelines import SUPPORTED_TASKS


In [ ]:
# Inspect supported tasks
rprint(SUPPORTED_TASKS)

In [ ]:
# Default models for each supported pipeline task
for task_name, task_info in SUPPORTED_TASKS.items():
    print(f"Task: {task_name}")

    model = task_info.get("default", {}).get("model")
    if model:
        print(f"[y] {model}")
    else:
        print("[x] No default model specified.")
    print("\n\n-------\n\n")


In [ ]:
# List models for a specific task (limit 5, sorted by downloads)
models = list(
    list_models(filter="text-generation", sort="downloads", direction=-1, limit=5)
)
for model in models:
    print(model.id)

# List ALL models for a specific task
# models = list(list_models(filter="text-generation", sort="downloads", direction=-1))
# print(len(models))
#  -> 291216, took ~120 seconds

In [ ]:
# Generate text (small model, can run on CPU)
generator = pipeline(task="text-generation", model="gpt2")
result = generator("Once upon a time")
rprint(result)

In [ ]:
# Stream generated text
streamer = TextStreamer(cast(AutoTokenizer, generator.tokenizer), skip_prompt=True)
generator(
    "1). Peel the tomato\n 2). Cut the onion.\n 3). Dice the cucumber.",
    max_length=50,
    streamer=streamer,
)

In [ ]:
# Sentiment analysis
classifier = pipeline(
    task="text-classification",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
)
result = classifier("I love this library!")
rprint(result)

In [ ]:
# Question answering
qa = pipeline(task="question-answering", model="distilbert-base-cased-distilled-squad")
question = "What is Hugging Face?"
context = """Hugging Face is a company that provides tools for natural language processing.
They created the transformers library.
They usually have a lot of fun.
They also dislike numbered lists and prefer bullet points."""

result = qa(question=question, context=context)
rprint(result)

In [ ]:
summarizer = pipeline(task="summarization", model="facebook/bart-large-cnn")
text = """
The Hugging Face transformers library provides thousands of pretrained models
to perform tasks on different modalities such as text, vision, and audio.
These models can be applied on text for tasks like classification, information
extraction, question answering, summarization, translation, and text generation
in over 100 languages. They also provide models for image classification,
object detection, and segmentation.
"""
result = summarizer(text, max_length=50, min_length=25)
rprint(result)